In [1]:
from Utils.ProjectPathsSetup import ProjectPathsSetup
ProjectPathsSetup().add_project_paths('./')

In [2]:
from DLArchitectures.EmbeddingLSTM import EmbeddingLSTM

In [3]:
model = EmbeddingLSTM(vo_size=500, embedding_size=50, lstm_size=100)

model = model.get_model()

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 title_embedding (Embedding)  (None, None, 50)         25000     
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 85,501
Trainable params: 85,501
Non-trainable params: 0
_________________________________________________________________


In [7]:
from DLArchitectures.SimpleDense import SimpleDense

model_s = SimpleDense(sents_length=20).get_model()


In [9]:
model_s.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 32)                672       
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________
